In [7]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

skills = pd.read_csv('mirana.txt',delimiter=';')
skills.loc[:,"Ratio"] = round(skills.Dmg/skills.Mana,1)
levels = skills.Level.unique()
skills.loc[:,"Ratio-Cd"] = skills["Ratio"]/skills["Cooldown"]

import json
import plotly.graph_objects as go

def plot_daños(df,heroe):
    layout = go.Layout(width=500,height=300,margin_b=5,title_text=heroe)
    mayor_ratio = df.sort_values(["RatioPegoPegan"],axis=0,ascending=False)
    x = mayor_ratio.index
    fig = go.Figure([go.Bar(x=x, y=mayor_ratio["%HPpego"],name="Pego"),
    go.Bar(x=x, y=mayor_ratio["%HPmePegan"],name="te Pegan"),
    go.Bar(x=x, y=mayor_ratio["RatioPegoPegan"],name="%")])
    fig.update_layout(layout)
    fig.show()

def plot_avanceLevels(dict_dfHeroes,valor,heroe_actual):
    layout = go.Layout(height=300,width=1000,margin_b=5,title_text=valor)
    levels = [x for x in range(1,21)]
    traces = [go.Scatter(x=levels,y=heroeDf.loc[valor,levels],name=heroeName) for heroeName,heroeDf in dict_dfHeroes.items()] #
    fig = go.Figure(data=traces)
    fig.update_layout(layout)
  
    fig.show()


df = pd.read_json('npc_heroes.json',dtype='float')
pd.set_option('display.max_rows', df.shape[0]-80)
df.drop(["HeroType","SoloDesire","RequiresBabysit","ProvidesBabysit","SurvivalRating","RequiresFarm","ProvidesSetup","RequiresSetup","BaseAttackSpeed","AttackAcquisitionRange","AttackRange","AttackCapabilities","StatusManaRegen","HeroID","Role","Rolelevels","Complexity","StatusHealthRegen","MovementSpeed"],axis=1,inplace=True)

#DF QUE SE USARA PARA CALCULAR EN BASE A UN HEROE
df_target_hero = df.copy()
df_target_hero.set_index("Heroe",inplace=True)
df_target_hero.loc[:,"BASE HP"] =    200 # HPBase
df_target_hero.loc[:,"BaseAttack"] = (df_target_hero["AttackDamageMin"]+df_target_hero["AttackDamageMax"])/2 # DMG PROMEDIO
print(df_target_hero)


def target_lvlUno(heroe,enemigos):
    """Esta funcion permite saber cuanto daño fisico pego y me pegan para un -heroe-"""
    daños = pd.DataFrame()
    daños["%HPpego"] = round(100*((df_target_hero.loc[heroe,"BaseAttack"]-df_target_hero.loc[enemigos,"ArmorPhysical"])/(df_target_hero.loc[enemigos,"BASE HP"])),1)  

    daños["%HPmePegan"] = round(100*((df_target_hero.loc[enemigos,"BaseAttack"]-df_target_hero.loc[heroe,"ArmorPhysical"])/df_target_hero.loc[heroe,"BASE HP"]),1)

    daños["RatioPegoPegan"] = round((daños["%HPpego"]/daños["%HPmePegan"]),1)
    plot_daños(daños,heroe)


#DF HEROES

def crearDfsEnemigos(heroe_actual,enemigos):
    enemigos.append(heroe_actual)
    
    dictDfHeroes = {}

    for hero in enemigos:
        dictDfHeroes[hero] = pd.DataFrame(index=["BASE ARMOR","MAIN ARMOR","DAMAGE MULT","HP","DMG","EHP"],columns=[x for x in range(1,21)])
        # ARMOR
        dictDfHeroes[hero].loc["BASE ARMOR",:] = df_target_hero.loc[hero,"ArmorPhysical"]
        dictDfHeroes[hero].loc["MAIN ARMOR",1] = dictDfHeroes[hero].loc["BASE ARMOR",1] + (df_target_hero.loc[hero,"Agility"]*0.16)
        dictDfHeroes[hero].loc["DAMAGE MULT",1] = 1 - ((0.052 * dictDfHeroes[hero].loc["MAIN ARMOR",1]) / (0.9 + 0.048 * abs(dictDfHeroes[hero].loc["MAIN ARMOR",1])))
        #HP

        dictDfHeroes[hero].loc["HP",1] = df_target_hero.loc[hero,"BASE HP"] +df_target_hero.loc[hero,"Strength"]*20 
        #ATK
        primary = df_target_hero.loc[hero,"Primary"].capitalize()

        for col in dictDfHeroes[hero].columns:
            if col == 1:
                pass
            else:
            # sven.loc["MAIN ARMOR",col] = sven.loc["BASE ARMOR",1] + (df_target_hero.loc["sven","Agility"]*0.16)
                dictDfHeroes[hero].loc["MAIN ARMOR",col] = (df_target_hero.loc[hero,"AgilityGain"]*0.16) + dictDfHeroes[hero].loc["MAIN ARMOR",col-1]
                dictDfHeroes[hero].loc["DAMAGE MULT",col] = 1 - ((0.052 * dictDfHeroes[hero].loc["MAIN ARMOR",col]) / (0.9 + 0.048 * abs(dictDfHeroes[hero].loc["MAIN ARMOR",col])))
                dictDfHeroes[hero].loc["HP",col] =  df_target_hero.loc[hero,"StrengthGain"]*20 + dictDfHeroes[hero].loc["HP",col-1]
            dictDfHeroes[hero].loc["DMG",col] = (df_target_hero.loc[hero,"BaseAttack"] + df_target_hero.loc[hero,f"{primary}Gain"]*col) #FALTARIA AGREGARLE DAMAGE MULT DEL ENEMIGO AL Q LE PEGA 

        dictDfHeroes[hero].loc["EHP"] = dictDfHeroes[hero].loc["HP"] /  dictDfHeroes[hero].loc["DAMAGE MULT"]
        # dictDfHeroes[hero].loc["RatioEHP/DMG"] = dictDfHeroes[hero].loc["DMG"]/dictDfHeroes[hero].loc["EHP"]

    # print(dictDfHeroes)
    plot_avanceLevels(dictDfHeroes,"DMG",heroe_actual)
    plot_avanceLevels(dictDfHeroes,"EHP",heroe_actual)

enemigos = ["bane","windrunner","sniper","pudge","phantom_lancer"]
hero_actual = "tiny"
# target_lvlUno(hero_actual,enemigos) 
crearDfsEnemigos(hero_actual,enemigos)



ArmorPhysical  AttackDamageMin  AttackDamageMax  AttackRate  \
Heroe                                                                         
antimage                 -1.0             29.0             33.0         1.4   
axe                      -1.0             27.0             31.0         1.7   
bane                      1.0             35.0             41.0         1.7   
bloodseeker               2.0             33.0             39.0         1.7   
crystal_maiden           -1.0             30.0             36.0         1.7   
...                       ...              ...              ...         ...   
dark_willow              -1.0             27.0             35.0         1.5   
grimstroke                0.0             21.0             25.0         1.7   
mars                     -1.0             29.0             37.0         1.8   
void_spirit              -1.0             32.0             36.0         1.7   
snapfire                  0.0             26.0             32.0     

In [62]:
for hero in df_target_hero.index.sort_values():
    print(hero)

abaddon
abyssal_underlord
alchemist
ancient_apparition
antimage
arc_warden
axe
bane
batrider
beastmaster
bloodseeker
bounty_hunter
brewmaster
bristleback
broodmother
centaur
chaos_knight
chen
clinkz
crystal_maiden
dark_seer
dark_willow
dazzle
death_prophet
disruptor
doom_bringer
dragon_knight
drow_ranger
earth_spirit
earthshaker
elder_titan
ember_spirit
enchantress
enigma
faceless_void
furion
grimstroke
gyrocopter
huskar
invoker
jakiro
juggernaut
keeper_of_the_light
kunkka
legion_commander
leshrac
lich
life_stealer
lina
lion
lone_druid
luna
lycan
magnataur
mars
medusa
meepo
mirana
monkey_king
morphling
naga_siren
necrolyte
nevermore
night_stalker
nyx_assassin
obsidian_destroyer
ogre_magi
omniknight
oracle
pangolier
phantom_assassin
phantom_lancer
phoenix
puck
pudge
pugna
queenofpain
rattletrap
razor
riki
rubick
sand_king
shadow_demon
shadow_shaman
shredder
silencer
skeleton_king
skywrath_mage
slardar
slark
snapfire
sniper
spectre
spirit_breaker
storm_spirit
sven
techies
templar_assassi